In [1]:
import xarray as xr

path = "../temp/*nc"
ds = xr.open_mfdataset( path, parallel = True )
ds

ValueError: zero-size array to reduction operation minimum which has no identity

In [5]:
import xarray as xr
path = "../temp/*nc"
ds = xr.open_mfdataset( path, parallel = True )
ds_i = []
for i in ds["DURACION"].values:
    df = ds.sel( DURACION = [i] ).to_dataframe().sort_values(
        ["south_north", "west_east", "Pcp"], ascending = [True, True, False] )
    df["m"] = ( list( range(1, ds["AÑO"].count().values + 1 ) )
        * ( ds["west_east"].max().values + 1 )
        * ( ds["south_north"].max().values + 1 ) )
    df["TIEMPO_RETORNO"] = ( ds["AÑO"].count().values + 1 ) / df["m"]
    df["PROBABILIDAD"] = 1/ df["TIEMPO_RETORNO"]
    ds_i.append( df.drop("m", axis = 1).to_xarray() )

ds_2 = xr.concat( ds_i, dim = "DURACION" ).to_dataframe().reset_index("AÑO"
    ).set_index( "TIEMPO_RETORNO", append = True ).reorder_levels(
    ["south_north", "west_east", "TIEMPO_RETORNO", "DURACION"] ).sort_index(
    ).rename( {"XLONG":"LONGITUD", "XLAT": "LATITUD", "Pcp":"INTENSIDAD"},
    axis = 1 ).to_xarray().set_coords( ["LONGITUD", "LATITUD"])
ds_2

<xarray.Dataset>
Dimensions:         (AÑO: 31, south_north: 112, west_east: 132, DURACION: 24)
Coordinates:
  * AÑO             (AÑO) int64 1985 1986 1987 1988 1989 ... 2012 2013 2014 2015
  * south_north     (south_north) int64 0 1 2 3 4 5 ... 106 107 108 109 110 111
  * west_east       (west_east) int64 0 1 2 3 4 5 6 ... 126 127 128 129 130 131
  * DURACION        (DURACION) int64 1 2 3 4 5 6 7 8 ... 17 18 19 20 21 22 23 24
Data variables:
    XLONG           (AÑO, south_north, west_east, DURACION) float32 -100.5 .....
    XLAT            (AÑO, south_north, west_east, DURACION) float32 18.14 ......
    Pcp             (AÑO, south_north, west_east, DURACION) float32 0.0 ... 0.0
    TIEMPO_RETORNO  (AÑO, south_north, west_east, DURACION) float64 6.4 ... 1...
    PROBABILIDAD    (AÑO, south_north, west_east, DURACION) float64 0.1562 .....

In [28]:
ds_2 = xr.concat( ds_i, dim = "DURACION" ).to_dataframe().reset_index("AÑO"
    ).set_index( "TIEMPO_RETORNO", append = True ).reorder_levels(
    ["south_north", "west_east", "TIEMPO_RETORNO", "DURACION"] ).sort_index(
    ).rename( {"XLONG":"LONGITUD", "XLAT": "LATITUD", "Pcp":"INTENSIDAD"},
    axis = 1 ).to_xarray().set_coords( ["LONGITUD", "LATITUD"] )

ds_2.to_netcdf( "../results/prec_miro_hist_hist.nc" )